# Using Real-Time Flight Fare Prediction to Find the Lowest Deals 

(working title)

**Authors**:
- Chris Kuzemka
- Irene Chau
- Ali Nazim
- Yurica Xu
- Aqib Rahim

# Table of Contents

Here goes the sections of this notebook breakdown and we can apply relevant authors to it as well. 

# Executive Summary

(write a short abstract of the project's total aim, brief important finding, and accomplishment)






## Data Overview

(actual dictionary goes here) -- Yurica's work

discussion on the data's integrity and composition 

#  Data Exploration and Cleaning

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


## Kaggle Itineraries

(discussion on limits of handling this data)

(discussion on its composition)

## JetBlue Focus

(discussion on why we went with this)

(discussion on problems existent still with its broad focus) <- showcasing of routes

(discussion on isolating NY models)

## Cleaning

(little nuances) <- nulls, useless features, 

(problems with labeling and constraints) <- made up information or missing information or inconsistent labeling lacking data

(outliers and why they are outliers)



## Feature Engineering and KPIs

(showoff some of the features created and plotting)

# Model Preprocessing

(re-discussion of any points on how the model might be influenced based on what we know)

(model encoding strategeis)



# Modeling

(preliminary models)

(discussion of observations and failures)

(showing off different models)

(discussion and nshowcasing of best models)

# Model Application

(discuss how the model is used in an applicable setting)

(consider discussions on how it can be improved)

(discuss how the scope could be increased to other airports and routes or markets if applicable)